# MGMTMSA 403 HW1

## Team Members: Yijie Fu, Guangying Pan

In [1]:
from gurobipy import *

In [2]:
# calculate target OR time for each department using the previous share
share = [0.484, 0.042, 0.253, 0.074, 0.053, 0.095]
t = [sh * 213.5 for sh in share]
t

[103.334, 8.967, 54.0155, 15.799, 11.3155, 20.2825]

In [3]:
departments = [0, 1, 2, 3, 4, 5]
rooms = [0, 1, 2, 3, 4]
days = [0, 1, 2, 3, 4]

# The availability of each room on each day of the week
tjk = {0: [9, 9, 9, 9, 7.5],
       1: [9, 9, 9, 9, 7.5],
       2: [9, 9, 9, 9, 7.5],
       3: [9, 9, 9, 9, 7.5],
       4: [9, 8, 8, 8, 6.5]}

### Question 1

In [4]:
# Construct a 'blank' model
mod = Model()

# Define decision variables
X = mod.addVars(departments, rooms, days, vtype=GRB.BINARY)
s = mod.addVars(departments)

# Construct constraints
for i in departments:
    mod.addConstr(s[i] >= t[i] - sum(X[i,j,k]*tjk[days[k]][j] for j in rooms for k in days))
    mod.addConstr(s[i] >= 0)

# only one department or no department can be in one room per day
for j in rooms:
    for k in days:
        mod.addConstr(sum(X[i,j,k] for i in departments) <= 1)

# Create the objective function, and set it to be minimized
mod.setObjective(sum(s[i]/t[i] for i in departments), GRB.MINIMIZE)

mod.update()

mod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 156 columns and 312 nonzeros
Model fingerprint: 0x3e32d60c
Variable types: 6 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 6 rows and 0 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 2.066116e-03, 50 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |

In [5]:
# Extract the solution status. 
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")

    # print optimized value obtained at the optimal solution
    print(f"\nOptimized si/ti: {mod.objval}")

    # Extract the optimal values of the decision variables
    for i in departments:
        print(f"\nDepartment {i+1}, Under-allocation: {s[i].X}")

    # print optimal schedule
    for k in days:
        for j in rooms:
            for i in departments:
                if X[i, j, k].X == 1:
                    print(f"Room {j+1}, Day {k+1}: Department {i+1}")

Solved to optimality

Optimized si/ti: 0.05190597648318124

Department 1, Under-allocation: 5.3340000000000005

Department 2, Under-allocation: 0.0

Department 3, Under-allocation: 0.015500000000002956

Department 4, Under-allocation: 0.0

Department 5, Under-allocation: 0.0

Department 6, Under-allocation: 0.0
Room 1, Day 1: Department 1
Room 2, Day 1: Department 1
Room 3, Day 1: Department 3
Room 4, Day 1: Department 1
Room 5, Day 1: Department 5
Room 1, Day 2: Department 1
Room 2, Day 2: Department 1
Room 3, Day 2: Department 3
Room 4, Day 2: Department 3
Room 5, Day 2: Department 6
Room 1, Day 3: Department 1
Room 2, Day 3: Department 2
Room 3, Day 3: Department 1
Room 4, Day 3: Department 3
Room 5, Day 3: Department 5
Room 1, Day 4: Department 1
Room 2, Day 4: Department 1
Room 3, Day 4: Department 3
Room 4, Day 4: Department 3
Room 5, Day 4: Department 6
Room 1, Day 5: Department 1
Room 2, Day 5: Department 4
Room 3, Day 5: Department 4
Room 4, Day 5: Department 1
Room 5, Day 5: 

### Question 2

In [6]:
# Construct a 'blank' model
mod = Model()

# new variable: floor
floors = [0, 1, 2]

# Define decision variable
X = mod.addVars(departments, rooms, days, vtype=GRB.BINARY)
s = mod.addVars(departments)

# Add new decision variable
Z = mod.addVars(departments, floors, days, vtype = GRB.BINARY)

# Construct constraints
for i in departments:
    mod.addConstr(s[i] >= t[i] - sum(X[i,j,k]*tjk[days[k]][j] for j in rooms for k in days))
    mod.addConstr(s[i] >= 0)

# only one department can be in one room per day
for j in rooms:
    for k in days:
        mod.addConstr(sum(X[i,j,k] for i in departments) <= 1)
        
# New constraints
# only one floor or no floor can be assigned to a department per day
for i in departments:
    for k in days:
        mod.addConstr(sum(Z[i,l,k] for l in floors) <= 1)

# floor 1: room1 + room2
# floor 2: room3 + room4
# floor 3: room5
for i in departments:
    for k in days:
        # when room1 or room2 is either/both occupied, floor1 = 1, otherwise floor1 = 0
        mod.addConstr(X[i,0,k] + X[i,1,k] >= Z[i,0,k])
        mod.addConstr(X[i,0,k] + X[i,1,k] <= 2*Z[i,0,k])
        
        # when room3 or room4 is either/both occupied, floor2 = 1, otherwise floor2 = 0
        mod.addConstr(X[i,2,k] + X[i,3,k] >= Z[i,1,k])
        mod.addConstr(X[i,2,k] + X[i,3,k] <= 2*Z[i,1,k])
        
        #  when room5 is occupied, floor3 = 1, otherwise floor3 = 0
        mod.addConstr(X[i,4,k] == Z[i,2,k])
        

# Create the objective function, and set it to be minimized
mod.setObjective(sum(s[i]/t[i] for i in departments), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.5.0 22F82)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 217 rows, 246 columns and 822 nonzeros
Model fingerprint: 0x6b506fb8
Variable types: 6 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Found heuristic solution: objective 6.0000000
Presolve removed 36 rows and 30 columns
Presolve time: 0.00s
Presolved: 181 rows, 216 columns, 756 nonzeros
Variable types: 4 continuous, 212 integer (210 binary)
Found heuristic solution: objective 5.9998574

Root relaxation: objective 1.387152e-01, 125 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [7]:
# Extract the solution status. 
if mod.status == GRB.OPTIMAL:
    print("Solved to optimality")

    # print optimized value obtained at the optimal solution
    print(f"\nOptimized si/ti: {mod.objval}")

    # Extract the optimal values of the decision variables
    for i in departments:
        print(f"\nDepartment {i+1}, Under-allocation: {s[i].X}")

    # print optimal schedule
    for k in days:
        for j in rooms:
            for i in departments:
                if X[i, j, k].X == 1:
                    print(f"Room {j+1}, Day {k+1}: Department {i+1}")

Solved to optimality

Optimized si/ti: 0.1387152340952639

Department 1, Under-allocation: 14.334

Department 2, Under-allocation: 0.0

Department 3, Under-allocation: 0.0

Department 4, Under-allocation: 0.0

Department 5, Under-allocation: 0.0

Department 6, Under-allocation: 0.0
Room 1, Day 1: Department 3
Room 2, Day 1: Department 3
Room 3, Day 1: Department 1
Room 4, Day 1: Department 1
Room 5, Day 1: Department 6
Room 1, Day 2: Department 1
Room 2, Day 2: Department 1
Room 3, Day 2: Department 3
Room 4, Day 2: Department 3
Room 5, Day 2: Department 5
Room 1, Day 3: Department 1
Room 2, Day 3: Department 1
Room 3, Day 3: Department 4
Room 4, Day 3: Department 4
Room 5, Day 3: Department 6
Room 1, Day 4: Department 1
Room 2, Day 4: Department 1
Room 3, Day 4: Department 3
Room 4, Day 4: Department 2
Room 5, Day 4: Department 6
Room 1, Day 5: Department 1
Room 2, Day 5: Department 1
Room 3, Day 5: Department 3
Room 4, Day 5: Department 3
Room 5, Day 5: Department 5
